# **1\.Análise de Dados: COVID-19 Dashboard**

# 1\.1 TDLR (resumo)

**1. Dashboard**
* Google Data Studio ([link](https://lookerstudio.google.com/reporting/83f3dffd-13dc-4cd3-a47b-737c46374db9))

**2. Fontes**
* Casos pela Unidade John Hopkins([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports)).

* Vacinação pela universidade de Oxford([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

# 1.2 Pandemia Coronavírus 2019

> A **COVID-19** é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: Governo brasileiro ([link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus)).

A disponibilidade de dados sobre a evolução da pandemia no tempo em uma determinada região geográfica é fundamental para o seu combate! Este projeto busca construir um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação do Brasil. O processamento de dados está neste [link](https://www.kaggle.com/nildomunizfrana/covid-dashboard/) e o dashboard, neste [link](https://lookerstudio.google.com/reporting/83f3dffd-13dc-4cd3-a47b-737c46374db9).


Os dados sobre casos da COVID-19 são compilados pelo centro de ciência de sistemas e engenharia da universidade americana John Hopkins ([link](https://www.jhu.edu/)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O website do projeto pode ser acessado neste [link](https://systems.jhu.edu/research/public-health/ncov/) enquanto os dados, neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

* **date:** data de referência;
* **state:** estado;
* **country:** país;
* **population:** população estimada;
* **confirmed:** número acumulado de infectados;
* **confirmed_1d:** número diário de infectados;
* **confirmed_moving_avg_7d:** média móvel de 7 dias do número diário de infectados;
* **confirmed_moving_avg_7d_rate_14d:** média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
* **deaths:** número acumulado de mortos;
* **deaths_1d:** número diário de mortos;
* **deaths_moving_avg_7d:** média móvel de 7 dias do número diário de mortos;
* **deaths_moving_avg_7d:** média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
* **month:** mês de referência;
* **year:** ano de referência.

Os dados sobre vacinação da COVID-19 são compilados pelo projeto Nosso Mundo em Dados (Our World in Data ou OWID) da universidade britânica de Oxford ([link](https://www.ox.ac.uk/)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de países. O website do projeto pode ser acessado neste [link](https://ourworldindata.org/) enquanto os dados, neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv). Abaixo estão descritos os dados derivados do seu processamento.

* **date:** data de referência;
* **country:** país;
* **population:** população estimada;
* **total:** número acumulado de doses administradas;
* **one_shot:** número acumulado de pessoas com uma dose;
* **one_shot_perc:** número acumulado relativo de pessoas com uma dose;
* **two_shots:** número acumulado de pessoas com duas doses;
* **two_shot_perc:** número acumulado relativo de pessoas com duas doses;
* **three_shots:** número acumulado de pessoas com três doses;
* **three_shot_perc:** número acumulado relativo de pessoas com três doses;
* **month:** mês de referência;
* **year:** ano de referência.

# 2. Pacotes e bibliotecas

In [4]:
import math
import numpy as np
import pandas as pd

from typing import Iterator
from datetime import datetime, timedelta

# 3. Extração

# 3.1 Casos
Compilação dos dados referentes ao caso se dá no formato aaaa/mm/dd (2021/12/01)

In [2]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [3]:
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


> **Definindo intervalo Portanto, precisamos iterar dentro de um intervalo de tempo definido para extraí-lo.**

In [5]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [ ]:
start_date = datetime(2021,  1,  1)
end_date  = datetime(2021, 12, 31)

>**De maneira iterativa vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.**

In [ ]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = cases.append(case, ignore_index=True)

In [ ]:
cases.query('Province_State == "Bahia"').head()

# **3.2 Vacinação**

In [ ]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

In [ ]:
vaccines.head()

> **Selecionando colunas de interesse e filtrando linhas referentes ao Brasil**

In [ ]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [ ]:
vaccines.head()

# **4. Transformação**

# **4.1 Casos**

In [ ]:
cases.shape

In [ ]:
cases.info()

In [ ]:
cases.head() 

> **Renomenado nome das colunas**

In [ ]:
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

> **Renomeando estados com acentos**

In [ ]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

# **4.1.1. Enriquecimento**

> **Vamos então computar novas colunas para enriquecer a base de daos**

> **Chaves temporais:**

In [ ]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

> **População estimada por estado:**

In [ ]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

> **Número, média móvel (7 dias) e estabilidade (14 dias) de casos e mortes por estado:**

In [ ]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status
  

for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = cases_.append(cases_per_state, ignore_index=True)

cases = cases_
cases_ = None

> **Garantindo os dtypes corretos**

In [ ]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

> **Reorganização das colunas e checagem**

In [ ]:
cases = cases[[
    'date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 
    'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d',
     'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 
     'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year'
]]

In [ ]:
cases.head(n=3)

# **4.2. Vacinação**

In [ ]:
vaccines.shape

In [ ]:
vaccines.info()

In [ ]:
vaccines.head()

> **Lidando com dados faltantes**

In [ ]:
vaccines = vaccines.fillna(method='ffill')

> **Equiparando os conjuntos de dados quanto ao tempo (granularidade)**

In [ ]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

> **Renomeando as colunas**

In [ ]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

# **4.2.1. Enriquecimento**

> **Novas colunas**

> **Chaves temporais:**

In [ ]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))     

> **Dados relativos:**

In [ ]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

> **Garantindo os dtypes corretos**

In [ ]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

> **Reorganização das colunas e checagem**

In [ ]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [ ]:
vaccines.tail(3)

# **5. Carregamento**

> **Casos:**

In [ ]:
# casos
cases.to_csv('./casos_covid_19.csv', sep=',', index=False)

 > **Vacinação:**

In [ ]:
# vacinação:
vaccines.to_csv('./vacinas_covid_19.csv', sep=',', index=False)

# **6. Dashboard**

O produto final desta análise se encontre em um dashboard do **Google Data Studio**. O dashboard de dados contem os seguintes elementos:

> **Key performance indicatornegrito (KPIs):**

1. Casos e mortes nas 24 horas;
2. Média móvel (7 dias) de casos e mortes;
3. Tendência de casos e mortes;
4. Proporção de vacinados com 1ª e 2ª dose.

> **Exploratory Data Analysis (EDA) interativa**
O dashboard de dados contém os seguintes gráficos para a análise exploraória de dados (exploratory data analysis ou EDA), interativa:

1. Distribuição do números de casos e mortes ao longo do tempo;
2. Distribuição da média móvel (7 dias) do números de casos e mortes ao longo do tempo;
3. Distribuição geográfica dos casos por estado por dia.

# **Covid Dashboard no Google Data Studio** ([link](https://lookerstudio.google.com/reporting/83f3dffd-13dc-4cd3-a47b-737c46374db9))

![](https://github.com/nildomuniz/ebac-analista-dados/blob/main/7.Data_Viz_BI/Module%2037%20-%203%C2%BA%20Projeto%20Dashboard%20de%20dados/1-covid_dashboad.PNG?raw=true)